# SoilGrids - Extraction des Propriétés du Sol

## C'est quoi SoilGrids ?

**SoilGrids** est une carte mondiale des propriétés du sol produite par ISRIC :
- Résolution spatiale de **250 mètres**
- Basée sur des modèles de machine learning
- Plusieurs profondeurs disponibles (0-5cm, 5-15cm, 15-30cm, etc.)

## Pourquoi c'est utile pour la qualité de l'eau ?

Les propriétés du sol affectent directement la chimie de l'eau :

| Propriété du sol | Impact sur la qualité de l'eau |
|------------------|--------------------------------|
| **pH du sol** | Influence le pH de l'eau, la solubilité des minéraux |
| **% Argiles** | Capacité de rétention des polluants, érosion |
| **Carbone organique** | Source de matière organique dissoute |
| **CEC** | Capacité d'échange des cations, alcalinité |

## Variables extraites

| Variable | Unité | Description |
|----------|-------|-------------|
| `soil_ph` | pH x 10 | pH du sol (0-5cm) |
| `soil_clay` | g/kg | Teneur en argile |
| `soil_sand` | g/kg | Teneur en sable |
| `soil_soc` | dg/kg | Carbone organique du sol |
| `soil_cec` | mmol(c)/kg | Capacité d'échange cationique |
| `soil_nitrogen` | cg/kg | Azote total |

## Source des données

**API REST ISRIC SoilGrids** : https://rest.isric.org

> Note: Planetary Computer n'héberge pas SoilGrids, on utilise donc l'API ISRIC directement.

---

## Étape 1 : Installation des dépendances

**Première exécution uniquement** : Après avoir exécuté cette cellule, il faut redémarrer le kernel.

In [11]:
!pip install uv
!uv pip install --system -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Using Python 3.13.1 environment at: c:\Program Files\Python313
Resolved 194 packages in 4.67s
error: Failed to install: jupyter_events-0.12.0-py3-none-any.whl (jupyter-events==0.12.0)
  Caused by: failed to create directory `c:\Program Files\Python313\Lib\site-packages\jupyter_events`: Accès refusé. (os error 5)


In [12]:
# =============================================================================
# IMPORTS
# =============================================================================

import warnings
warnings.filterwarnings("ignore")

# Manipulation de données
import numpy as np
import pandas as pd

# Requêtes HTTP pour l'API SoilGrids
import requests
import time

from tqdm import tqdm  # Barre de progression
import os

print("Imports OK!")

Imports OK!


---

## Étape 2 : Définition des constantes et fonctions

### Variables SoilGrids à extraire

In [13]:
# =============================================================================
# CONSTANTES
# =============================================================================

# URL de base de l'API SoilGrids
SOILGRIDS_API_URL = "https://rest.isric.org/soilgrids/v2.0/properties/query"

# Propriétés à extraire (nom API -> nom feature)
SOILGRIDS_PROPERTIES = {
    'phh2o': 'soil_ph',       # pH du sol (x10)
    'clay': 'soil_clay',      # Teneur en argile (g/kg)
    'sand': 'soil_sand',      # Teneur en sable (g/kg)
    'soc': 'soil_soc',        # Carbone organique (dg/kg)
    'cec': 'soil_cec',        # Capacité échange cationique (mmol/kg)
    'nitrogen': 'soil_nitrogen',  # Azote total (cg/kg)
}

# Profondeur à extraire
DEPTH = '0-5cm'

# Dossier de sortie
OUTPUT_DIR = "../data/processed"

print(f"Variables SoilGrids à extraire :")
for api_name, feature_name in SOILGRIDS_PROPERTIES.items():
    print(f"  - {feature_name} (API: {api_name})")
print(f"\nProfondeur: {DEPTH}")

Variables SoilGrids à extraire :
  - soil_ph (API: phh2o)
  - soil_clay (API: clay)
  - soil_sand (API: sand)
  - soil_soc (API: soc)
  - soil_cec (API: cec)
  - soil_nitrogen (API: nitrogen)

Profondeur: 0-5cm


### Fonction : Extraire les propriétés du sol via l'API REST

In [14]:
def extract_soilgrids_properties(lat, lon, properties=None, depth='0-5cm', debug=False):
    """
    Extrait les propriétés du sol pour un point via l'API ISRIC SoilGrids.
    
    Paramètres:
        lat, lon : coordonnées du point
        properties : liste des propriétés à extraire (défaut: toutes)
        depth : profondeur (ex: '0-5cm', '5-15cm', '15-30cm')
        debug : afficher les messages de débogage
    
    Retourne:
        dict avec les valeurs de chaque propriété
    """
    if properties is None:
        properties = list(SOILGRIDS_PROPERTIES.keys())
    
    # Initialiser les résultats
    results = {SOILGRIDS_PROPERTIES[p]: np.nan for p in properties}
    
    try:
        # Construire l'URL avec les paramètres
        params = {
            'lon': lon,
            'lat': lat,
            'property': properties,
            'depth': [depth],
            'value': ['mean']  # On prend la valeur moyenne
        }
        
        if debug:
            print(f"  Requête API: lat={lat}, lon={lon}")
        
        # Faire la requête
        response = requests.get(SOILGRIDS_API_URL, params=params, timeout=30)
        
        if debug:
            print(f"  Status: {response.status_code}")
        
        if response.status_code != 200:
            if debug:
                print(f"  ⚠️ Erreur HTTP: {response.status_code}")
            return results
        
        data = response.json()
        
        if debug:
            print(f"  Réponse reçue: {list(data.keys())}")
        
        # Extraire les valeurs
        if 'properties' in data and 'layers' in data['properties']:
            for layer in data['properties']['layers']:
                prop_name = layer['name']
                if prop_name in SOILGRIDS_PROPERTIES:
                    feature_name = SOILGRIDS_PROPERTIES[prop_name]
                    
                    # Chercher la valeur pour la profondeur demandée
                    for depth_data in layer['depths']:
                        if depth_data['label'] == depth:
                            value = depth_data['values'].get('mean')
                            if value is not None:
                                results[feature_name] = float(value)
                            if debug:
                                print(f"  {feature_name}: {value}")
                            break
        
    except requests.exceptions.Timeout:
        if debug:
            print("  ⚠️ Timeout")
    except Exception as e:
        if debug:
            print(f"  ❌ Erreur: {type(e).__name__}: {e}")
    
    return results

### Test de diagnostic

In [15]:
# =============================================================================
# TEST DE DIAGNOSTIC
# =============================================================================

print("Test de diagnostic SoilGrids (API ISRIC)")
print("=" * 50)

# Point de test en Afrique du Sud
test_lat, test_lon = -26.45, 28.085833

print(f"\nTest avec le point: lat={test_lat}, lon={test_lon}")
print("-" * 50)

# Extraire avec debug activé
result = extract_soilgrids_properties(test_lat, test_lon, depth=DEPTH, debug=True)

print("\n" + "=" * 50)
print("Résultats:")
for k, v in result.items():
    if pd.notna(v):
        print(f"  {k}: {v}")

Test de diagnostic SoilGrids (API ISRIC)

Test avec le point: lat=-26.45, lon=28.085833
--------------------------------------------------
  Requête API: lat=-26.45, lon=28.085833
  Status: 200
  Réponse reçue: ['type', 'geometry', 'properties', 'query_time_s']
  soil_cec: 218
  soil_clay: 260
  soil_nitrogen: 144
  soil_ph: 61
  soil_sand: 627
  soil_soc: 180

Résultats:
  soil_ph: 61.0
  soil_clay: 260.0
  soil_sand: 627.0
  soil_soc: 180.0
  soil_cec: 218.0
  soil_nitrogen: 144.0


---

## Étape 3 : Extraction pour les données d'entraînement

In [16]:
# Charger les données
Water_Quality_df = pd.read_csv("../data/raw/water_quality_training_dataset.csv")

print(f"Nombre d'observations : {len(Water_Quality_df)}")

# Sites uniques
training_sites = Water_Quality_df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
print(f"Sites uniques à traiter : {len(training_sites)}")

Nombre d'observations : 9319
Sites uniques à traiter : 162


In [17]:
# =============================================================================
# EXTRACTION SOILGRIDS - TRAINING
# =============================================================================

print(f"Extraction pour {len(training_sites)} sites uniques...")
print("⚠️ L'API ISRIC a une limite de requêtes - on ajoute un délai entre chaque requête")

# Fichier de sauvegarde incrémentale
BACKUP_PATH = "../data/processed/soilgrids_training_backup.csv"
SAVE_EVERY = 50

print(f"Sauvegarde automatique tous les {SAVE_EVERY} sites\n")

training_results = []
completed_count = 0

for idx, row in tqdm(training_sites.iterrows(), total=len(training_sites), desc="Extraction"):
    lat, lon = row['Latitude'], row['Longitude']
    
    # Extraire toutes les propriétés
    soil_props = extract_soilgrids_properties(lat, lon, depth=DEPTH, debug=False)
    
    # Ajouter les coordonnées
    result = {'Latitude': lat, 'Longitude': lon}
    result.update(soil_props)
    
    training_results.append(result)
    completed_count += 1
    
    # Petit délai pour ne pas surcharger l'API
    time.sleep(0.5)
    
    # Sauvegarde incrémentale
    if completed_count % SAVE_EVERY == 0:
        backup_df = pd.DataFrame(training_results)
        backup_df.to_csv(BACKUP_PATH, index=False)
        print(f"\n💾 Sauvegarde : {completed_count}/{len(training_sites)} sites")

# Sauvegarde finale
training_soilgrids_unique = pd.DataFrame(training_results)
training_soilgrids_unique.to_csv(BACKUP_PATH, index=False)

print(f"\n✅ Extraction terminée : {len(training_soilgrids_unique)} sites")

Extraction pour 162 sites uniques...
⚠️ L'API ISRIC a une limite de requêtes - on ajoute un délai entre chaque requête
Sauvegarde automatique tous les 50 sites



Extraction:  31%|███       | 50/162 [00:52<01:43,  1.08it/s]


💾 Sauvegarde : 50/162 sites


Extraction:  62%|██████▏   | 100/162 [01:42<00:53,  1.16it/s]


💾 Sauvegarde : 100/162 sites


Extraction:  93%|█████████▎| 150/162 [02:30<00:15,  1.28s/it]


💾 Sauvegarde : 150/162 sites


Extraction: 100%|██████████| 162/162 [02:40<00:00,  1.01it/s]


✅ Extraction terminée : 162 sites


In [18]:
# Fusionner avec le DataFrame original
training_soilgrids_df = Water_Quality_df[['Latitude', 'Longitude', 'Sample Date']].merge(
    training_soilgrids_unique,
    on=['Latitude', 'Longitude'],
    how='left'
)

print(f"DataFrame final : {len(training_soilgrids_df)} lignes")

DataFrame final : 9319 lignes


In [19]:
# Sauvegarder le fichier CSV
output_path = os.path.join(OUTPUT_DIR, 'soilgrids_features_training.csv')
training_soilgrids_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

Fichier créé : ../data/processed\soilgrids_features_training.csv


In [20]:
# Aperçu des données extraites
print("Aperçu des données extraites :")
print(f"- Lignes : {len(training_soilgrids_df)}")
print(f"- Colonnes : {list(training_soilgrids_df.columns)}")

# Statistiques
soil_cols = [col for col in training_soilgrids_df.columns if col.startswith('soil_')]
print(f"\nStatistiques des propriétés du sol :")
print(training_soilgrids_df[soil_cols].describe())

display(training_soilgrids_df.head())

Aperçu des données extraites :
- Lignes : 9319
- Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'soil_ph', 'soil_clay', 'soil_sand', 'soil_soc', 'soil_cec', 'soil_nitrogen']

Statistiques des propriétés du sol :
           soil_ph    soil_clay    soil_sand     soil_soc     soil_cec  \
count  6401.000000  6401.000000  6401.000000  6401.000000  6401.000000   
mean     65.271208   237.641775   587.643181   238.164974   194.000937   
std       4.736546    60.418741   115.546808   138.309351    26.876064   
min      56.000000    63.000000   286.000000    94.000000    97.000000   
25%      62.000000   207.000000   528.000000   148.000000   177.000000   
50%      65.000000   234.000000   601.000000   192.000000   193.000000   
75%      68.000000   281.000000   663.000000   266.000000   209.000000   
max      82.000000   417.000000   838.000000   846.000000   279.000000   

       soil_nitrogen  
count    6401.000000  
mean      176.952195  
std        72.509382  
min        45.000000  
2

,Latitude,Longitude,Sample Date,soil_ph,soil_clay,soil_sand,soil_soc,soil_cec,soil_nitrogen
0,-28.760833,17.730278,02-01-2011,82.0,69.0,838.0,546.0,166.0,141.0
1,-26.861111,28.884722,03-01-2011,65.0,302.0,540.0,161.0,200.0,200.0
2,-26.450000,28.085833,03-01-2011,61.0,260.0,627.0,180.0,218.0,144.0
3,-27.671111,27.236944,03-01-2011,68.0,264.0,616.0,162.0,195.0,129.0
4,-27.356667,27.286389,03-01-2011,67.0,245.0,637.0,150.0,186.0,132.0


---

## Étape 4 : Extraction pour les données de validation

In [21]:
# Charger le template de soumission
Validation_df = pd.read_csv('../data/raw/submission_template.csv')

print(f"Nombre de sites de validation : {len(Validation_df)}")

# Sites uniques
validation_sites = Validation_df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
print(f"Sites uniques à traiter : {len(validation_sites)}")

Nombre de sites de validation : 200
Sites uniques à traiter : 24


In [22]:
# =============================================================================
# EXTRACTION SOILGRIDS - VALIDATION
# =============================================================================

print(f"Extraction pour {len(validation_sites)} sites uniques...")

BACKUP_PATH_VAL = "../data/processed/soilgrids_validation_backup.csv"
SAVE_EVERY_VAL = 20

validation_results = []
completed_count = 0

for idx, row in tqdm(validation_sites.iterrows(), total=len(validation_sites), desc="Extraction"):
    lat, lon = row['Latitude'], row['Longitude']
    
    soil_props = extract_soilgrids_properties(lat, lon, depth=DEPTH, debug=False)
    
    result = {'Latitude': lat, 'Longitude': lon}
    result.update(soil_props)
    
    validation_results.append(result)
    completed_count += 1
    
    # Petit délai pour ne pas surcharger l'API
    time.sleep(0.5)
    
    if completed_count % SAVE_EVERY_VAL == 0:
        backup_df = pd.DataFrame(validation_results)
        backup_df.to_csv(BACKUP_PATH_VAL, index=False)
        print(f"\n💾 Sauvegarde : {completed_count}/{len(validation_sites)} sites")

# Sauvegarde finale
validation_soilgrids_unique = pd.DataFrame(validation_results)
validation_soilgrids_unique.to_csv(BACKUP_PATH_VAL, index=False)

print(f"\n✅ Extraction terminée : {len(validation_soilgrids_unique)} sites")

Extraction pour 24 sites uniques...


Extraction:  83%|████████▎ | 20/24 [00:20<00:03,  1.05it/s]


💾 Sauvegarde : 20/24 sites


Extraction: 100%|██████████| 24/24 [00:24<00:00,  1.03s/it]


✅ Extraction terminée : 24 sites


In [23]:
# Fusionner avec le DataFrame original
validation_soilgrids_df = Validation_df[['Latitude', 'Longitude', 'Sample Date']].merge(
    validation_soilgrids_unique,
    on=['Latitude', 'Longitude'],
    how='left'
)

print(f"DataFrame final : {len(validation_soilgrids_df)} lignes")

DataFrame final : 200 lignes


In [24]:
# Sauvegarder le fichier CSV
output_path = os.path.join(OUTPUT_DIR, 'soilgrids_features_validation.csv')
validation_soilgrids_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

Fichier créé : ../data/processed\soilgrids_features_validation.csv


In [25]:
# Aperçu des données de validation
print(f"Données de validation : {len(validation_soilgrids_df)} lignes")
print(f"\nStatistiques :")
print(validation_soilgrids_df[soil_cols].describe())

display(validation_soilgrids_df.head())

Données de validation : 200 lignes

Statistiques :
          soil_ph   soil_clay   soil_sand    soil_soc    soil_cec  \
count  165.000000  165.000000  165.000000  165.000000  165.000000   
mean    64.381818  240.642424  500.242424  334.454545  215.787879   
std      4.108992   18.200426   52.982527   97.764990   29.271185   
min     57.000000  156.000000  404.000000  195.000000  172.000000   
25%     61.000000  233.000000  478.000000  253.000000  196.000000   
50%     63.000000  235.000000  517.000000  350.000000  200.000000   
75%     66.000000  249.000000  534.000000  374.000000  244.000000   
max     71.000000  281.000000  658.000000  577.000000  279.000000   

       soil_nitrogen  
count     165.000000  
mean      272.163636  
std        88.194027  
min       136.000000  
25%       194.000000  
50%       319.000000  
75%       320.000000  
max       437.000000  


,Latitude,Longitude,Sample Date,soil_ph,soil_clay,soil_sand,soil_soc,soil_cec,soil_nitrogen
0,-32.043333,27.822778,01-09-2014,66.0,249.0,517.0,257.0,196.0,194.0
1,-33.329167,26.077500,16-09-2015,65.0,222.0,583.0,368.0,246.0,381.0
2,-32.991639,27.640028,07-05-2015,60.0,234.0,478.0,374.0,200.0,319.0
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,62.0,252.0,404.0,399.0,172.0,246.0


---

## Résumé

**Ce qu'on a fait :**
1. Connecté à Microsoft Planetary Computer
2. Pour chaque site de mesure, extrait les propriétés du sol (couche 0-5cm)
3. Créé 2 fichiers CSV avec les features du sol

**Features extraites :**

| Feature | Description | Impact attendu |
|---------|-------------|----------------|
| soil_ph | pH du sol | Alcalinité, conductivité |
| soil_clay | % argile | Rétention polluants |
| soil_sand | % sable | Drainage, infiltration |
| soil_soc | Carbone organique | Matière organique dissoute |
| soil_cec | Capacité échange cationique | Alcalinité, minéraux |
| soil_nitrogen | Azote total | Nutriments |

**Fichiers créés :**

| Fichier | Description |
|---------|-------------|
| soilgrids_features_training.csv | Features pour l'entraînement |
| soilgrids_features_validation.csv | Features pour la validation |

**Prochaine étape :**
- Extraire les données DEM (altitude, pente)
- Intégrer toutes les features dans le modèle